#Multi-objective Probe Placement Optimization
This is based on a paper by Lou et al. (GT2020-14867 and GT2020-15465), Reconstructing Compressor Non-Uniform Circumferential Flow Field
from Spatially Undersampled Data.

##Problem Description
The flow in turbomachinery experiments is inherently nonuniform around the annulus, and understanding the circumferential variation of the flowfield is often critical to the technological advancement of both compressors and turbines. However, characterizing the circumferential variation of the flow field with high spatial resolution is often impractical: physical constraints limit the instrumentation that can be placed in the flow path, and circumferential probe traverses are costly and complex.

##Approach
There are two applications of circumferential signal reconstruction:
  1) Rig design (spefically, probe placement)
  2) Data analysis

Concerning rig design, the primary wavenumbers of interest must be determined. These may be provide in detail from 

###Step 1: Cloning the Project (House keeping)
Lets clone the test project in Glennopt. We will need the test folder located in GlennOPT/test/ProbePlacement_multi/parallel

In [6]:
# Clone the source code for GlennOPT
!git clone https://github.com/nasa/GlennOPT.git
# Little Housekeeping
!cp -r GlennOPT/test/ProbePlacement_multi/parallel/Evaluation/ .    # Copy the folder we need 
!rm GlennOPT/ -r        # Deletes GlennOPT source code. We don't need this anymore
!rm sample_data -r


fatal: destination path 'GlennOPT' already exists and is not an empty directory.


In [19]:
# Install GlennOPT 
!python --version
!pip install glennopt

Python 3.6.9
     |████████████████████████████████| 11.6MB 359kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Import relevant libraries

In [5]:
from glennopt.helpers import Parameter, parallel_settings, de_mutation_type, mutation_parameters
from glennopt.optimizers import NSGA3
import numpy as np
import os

In [6]:
# Define evaluation parameters 
nProbes = 10
minSpacing = 3
probeSpacing = 360/nProbes
tLo     = np.zeros(nProbes)
tHi     = np.zeros(nProbes)
eval_parameters = list()
for i in range(nProbes):
    tLo[i] = probeSpacing*i
    if i != nProbes-1:
        tHi[i] = probeSpacing*(i+1) - minSpacing
    else:
        tHi[-1] = probeSpacing*(i+1)    
    eval_parameters.append(Parameter(name="x"+str(i+1),min_value=tLo[i],max_value=tHi[i]))
constraints = (tLo,tHi)

# Define the number of objectives
objectives = list()
objectives.append(Parameter(name='objective1'))
objectives.append(Parameter(name='objective2'))

# Define any performance parameters you want to keep track of (tracking only)
perf_parameters = list()
perf_parameters.append(Parameter(name='PearsonR'))
perf_parameters.append(Parameter(name='RMS_Error'))

In [7]:
# Set up the optimizer
current_dir = os.getcwd()
pop_size = 48
ns = NSGA3(eval_script = "Evaluation/evaluation.py", eval_folder="Evaluation",pop_size=pop_size,optimization_folder=current_dir)
ns.add_eval_parameters(eval_params = eval_parameters)
ns.add_performance_parameters(perf_parameters)
ns.add_objectives(objectives=objectives)

Enable Parallel Execution (OPTIONAL) 

In [8]:
# Parallel Settings (You don't need to run this block if you only want serial execution)
ns.parallel_settings.concurrent_executions = 8    # Change to 1 for serial
ns.parallel_settings.cores_per_execution: 1    
ns.parallel_settings.execution_timeout = 0.2      # minutes

##Run the Design of Experiments
Design of experiments is used to sample the evaluation space. Say you have 5 variables and f(x[1-5]) = y[1,2] and each x1 through x5 have min and max bounds. The design of experiments is used to evaluate different combinations of x1 to x5 which are used as the starting population (pop_start=-1) 

In [9]:
# Run the Design of Experiments
ns.start_doe(doe_size=128)                                    # This generates individuals that have evaluation parameters randomly distributed between the bounds 


In [ ]:
# Execute the Optimization
ns.optimize_from_population(pop_start=-1,n_generations=80)    # Start from the DOE and iterate from pop 0 to 79